In [3]:
import pandas as pd

urla1 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis/Datasets/menu_engineering_desc_cleaned.csv"

url1a2 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis/Datasets/menu_engineering_dogs.csv"

urla3 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis/Datasets/menu_engineering_input_items.csv"

urla4 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/item_addon_association.csv"

urla5 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/item_addon_association.csv"

urla6 ="https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/item_item_associations.csv"

analysis1  = pd.read_csv(urla1)
analysis2  = pd.read_csv(url1a2)
analysis3  = pd.read_csv(urla3)
analysis4  = pd.read_csv(urla4)
item_add = pd.read_csv(urla5)
item_item = pd.read_csv(urla6)



In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report




url_menu_items = (
   "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis/Datasets/menu_engineering_input_items.csv"

)

df = pd.read_csv(url_menu_items)

print("Columns in dataset:")
print(df.columns.tolist())


#  BASIC CLEANING


# Required columns for menu engineering
required_cols = ["price", "purchases"]

df = df.dropna(subset=required_cols)

# Ensure numeric
df["price"] = pd.to_numeric(df["price"], errors="coerce")
df["purchases"] = pd.to_numeric(df["purchases"], errors="coerce")

df = df.dropna(subset=required_cols)


#  FEATURE ENGINEERING (BUSINESS-ALIGNED)


# Revenue
df["revenue"] = df["price"] * df["purchases"]

# Estimated cost (40% food cost assumption)
df["estimated_cost"] = df["price"] * 0.40

# Contribution margin
df["margin"] = df["price"] - df["estimated_cost"]


# CREATE MENU ENGINEERING LABELS (GROUND TRUTH)


popularity_threshold = df["purchases"].median()
profit_threshold = df["margin"].median()

def menu_engineering_label(row):
    if row["purchases"] >= popularity_threshold and row["margin"] >= profit_threshold:
        return "Star"
    elif row["purchases"] >= popularity_threshold and row["margin"] < profit_threshold:
        return "Plowhorse"
    elif row["purchases"] < popularity_threshold and row["margin"] >= profit_threshold:
        return "Puzzle"
    else:
        return "Dog"

df["menu_label"] = df.apply(menu_engineering_label, axis=1)

print("\nMenu category distribution:")
print(df["menu_label"].value_counts())

# PREPARE DATA FOR CLASSIFICATION MODEL


features = ["price", "purchases", "margin"]
X = df[features]
y = df["menu_label"]

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Train / test split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y_encoded,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded
)

# TRAIN CLASSIFICATION MODEL

model = DecisionTreeClassifier(
    max_depth=4,
    random_state=42
)

model.fit(X_train, y_train)

#  EVALUATION
y_pred = model.predict(X_test)

print("\nClassification Report:")
print(classification_report(
    y_test,
    y_pred,
    target_names=label_encoder.classes_
))




Columns in dataset:
['item_id', 'item_name', 'price', 'purchases', 'revenue', 'profit', 'category', 'description']

Menu category distribution:
menu_label
Plowhorse    16827
Puzzle       16390
Star         16282
Dog          15814
Name: count, dtype: int64

Classification Report:
              precision    recall  f1-score   support

         Dog       1.00      1.00      1.00      3163
   Plowhorse       1.00      1.00      1.00      3366
      Puzzle       1.00      1.00      1.00      3278
        Star       1.00      1.00      1.00      3256

    accuracy                           1.00     13063
   macro avg       1.00      1.00      1.00     13063
weighted avg       1.00      1.00      1.00     13063



In [7]:
import  pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import json
from datetime import datetime
import warnings

urla1 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis/Datasets/menu_engineering_desc_cleaned.csv"

url1a2 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis/Datasets/menu_engineering_dogs.csv"

urla3 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis/Datasets/menu_engineering_input_items.csv"

urla4 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/item_addon_association.csv"

urla5 = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/item_addon_association.csv"

urla6 ="https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/item_item_associations.csv"

df_items = pd.read_csv(urla3)
df_item_assoc = pd.read_csv(urla6)
df_addon_assoc = pd.read_csv(urla5)
df_desc = pd.read_csv(urla1)
df_whatif = pd.read_csv(urla4)

warnings.filterwarnings('ignore')

class MenuEngineeringSystem:
    def __init__(self):
        """Initialize the Menu Engineering System with ML model"""
        self.model = DecisionTreeClassifier(max_depth=5, random_state=42)
        self.label_encoder = LabelEncoder()
        self.df_items = pd.DataFrame()
        self.ingredient_costs = pd.DataFrame()
        self.associations = pd.DataFrame()
        self.trained = False

    def load_data_from_files(self, addon_file = urla6,
                              item_file = urla6):

        print("Loading data from files...")

        try:
            # Load association files
            df_item_assoc = pd.read_csv(urla6)
            df_addon_assoc = pd.read_csv(urla5)

            # Create synthetic menu items for training based on associations
            # This simulates a real menu with categories
            unique_items = set()
            for col in ['antecedent', 'consequent']:
                if col in df_addon_assoc.columns:
                    unique_items.update(df_addon_assoc[col].unique())

            # Create training data
            items_list = []
            categories = ['Star', 'Plowhorse', 'Puzzle', 'Dog']

            for i, item in enumerate(list(unique_items)[:50]):  # Limit for training
                category = categories[i % 4]

                # Simulate realistic metrics based on category
                if category == 'Star':
                    price = np.random.uniform(12, 20)
                    purchases = np.random.randint(150, 300)
                    food_cost = price * 0.3
                elif category == 'Plowhorse':
                    price = np.random.uniform(8, 15)
                    purchases = np.random.randint(100, 250)
                    food_cost = price * 0.45
                elif category == 'Puzzle':
                    price = np.random.uniform(15, 25)
                    purchases = np.random.randint(20, 80)
                    food_cost = price * 0.35
                else:  # Dog
                    price = np.random.uniform(7, 12)
                    purchases = np.random.randint(10, 50)
                    food_cost = price * 0.50

                items_list.append({
                    'item_id': i,
                    'item_name': item,
                    'price': round(price, 2),
                    'food_cost': round(food_cost, 2),
                    'purchases': purchases,
                    'category': category,
                    'desc_length': np.random.randint(20, 80)
                })

            self.menu_items = pd.DataFrame(items_list)

            self.associations = {
                'addon': df_addon_assoc,
                'item': df_item_assoc
            }

            print(f"✓ Loaded {len(self.menu_items)} training items")
            print(f"✓ Loaded {len(df_addon_assoc)} add-on associations")
            print(f"✓ Loaded {len(df_item_assoc)} item-to-item associations")

            return True

        except Exception as e:
            print(f"✗ Error loading data: {e}")
            import traceback
            traceback.print_exc()
            return False

    def train_model(self):
        """Train the ML model on menu categorization"""
        print("\nTraining ML model...")

        if len(self.menu_items) == 0:
            print("✗ No data loaded. Please load data first.")
            return False

        # Prepare features
        features = ['price', 'purchases', 'desc_length']
        X = self.menu_items[features]
        y = self.menu_items['category']

        # Encode categories
        y_encoded = self.label_encoder.fit_transform(y)

        # Split data for validation
        X_train, X_test, y_train, y_test = train_test_split(
            X, y_encoded, test_size=0.2, random_state=42
        )

        # Train model
        self.model.fit(X_train, y_train)

        # Validate
        y_pred = self.model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        print(f"✓ Model trained successfully")
        print(f"✓ Accuracy: {accuracy*100:.2f}%")
        print(f"\nCategory distribution:")
        print(pd.Series(self.label_encoder.inverse_transform(y_encoded)).value_counts())

        self.trained = True
        return True

    def analyze_single_item(self, item_data):
        """
        Analyze a single menu item

        Parameters:
        -----------
        item_data : dict
            Dictionary with keys: item_name, price, food_cost, purchases, desc_length
        """
        if not self.trained:
            print(" Model not trained. Please train the model first.")
            return None

        # Prepare features
        features = np.array([[
            item_data['price'],
            item_data['purchases'],
            item_data.get('desc_length', 0)
        ]])

        # Predict category
        category_encoded = self.model.predict(features)[0]
        category = self.label_encoder.inverse_transform([category_encoded])[0]

        # Calculate metrics
        contribution_margin = item_data['price'] - item_data['food_cost']
        food_cost_percentage = (item_data['food_cost'] / item_data['price']) * 100

        # Calculate popularity index
        total_purchases = self.menu_items['purchases'].sum()
        popularity = (item_data['purchases'] / total_purchases) * 100

        # Calculate profitability metrics
        avg_cm = self.menu_items.apply(
            lambda row: (row['price'] - row.get('food_cost', row['price'] * 0.3)),
            axis=1
        ).mean()

        profit_factor = (contribution_margin / avg_cm) * 100

        return {
            'item_name': item_data['item_name'],
            'category': category,
            'price': item_data['price'],
            'food_cost': item_data['food_cost'],
            'contribution_margin': contribution_margin,
            'food_cost_percentage': food_cost_percentage,
            'purchases': item_data['purchases'],
            'popularity': popularity,
            'profit_factor': profit_factor,
            'recommendation': self._generate_recommendation(category, item_data)
        }

    def _generate_recommendation(self, category, item_data):
        """Generate strategic recommendations based on category"""
        recommendations = {
            'Star': {
                'action': 'PROTECT & PROMOTE',
                'strategy': [
                    'Feature prominently on menu (top-right corner)',
                    'Train staff to actively recommend this item',
                    'Maintain consistent quality',
                    'Consider slight price increase to maximize profit',
                    'Use as anchor for combo deals'
                ],
                'description_advice': 'Keep description concise (30-50 characters) to reduce decision fatigue',
                'upsell': 'Pair with high-margin add-ons'
            },
            'Plowhorse': {
                'action': 'RE-ENGINEER FOR PROFIT',
                'strategy': [
                    'Increase price by 5-10% (demand is proven)',
                    'Reduce portion size slightly to lower costs',
                    'Replace expensive ingredients with quality alternatives',
                    'Bundle with low-cost, high-margin sides',
                    'Optimize recipe for cost efficiency'
                ],
                'description_advice': 'Shorten description to emphasize value',
                'upsell': 'Cross-sell with Stars to increase basket value'
            },
            'Puzzle': {
                'action': 'INCREASE VISIBILITY',
                'strategy': [
                    'Reposition to high-visibility menu area',
                    'Add appealing photo or illustration',
                    'Rename with more enticing, descriptive name',
                    'Feature as "Chef\'s Special" or "Hidden Gem"',
                    'Bundle with popular items (Stars)',
                    'Offer limited-time promotion to drive trials'
                ],
                'description_advice': 'Expand description to highlight unique value proposition',
                'upsell': 'Create combo with Star items'
            },
            'Dog': {
                'action': 'ELIMINATE OR TRANSFORM',
                'strategy': [
                    'Remove from menu if no improvement in 2 months',
                    'Complete recipe overhaul if keeping',
                    'Reduce to smallest menu presence',
                    'Use only as loss leader if strategic value exists',
                    'Replace with customer-requested alternatives'
                ],
                'description_advice': 'Not worth optimizing - consider removal',
                'upsell': 'Do not promote - waste of marketing effort'
            }
        }

        base_rec = recommendations.get(category, recommendations['Dog'])

        # Add specific price recommendations
        if category == 'Plowhorse':
            current_profit = item_data['price'] - item_data['food_cost']
            suggested_price = item_data['price'] * 1.05
            projected_profit = suggested_price - item_data['food_cost']
            base_rec['price_simulation'] = {
                'current_price': item_data['price'],
                'suggested_price': round(suggested_price, 2),
                'current_profit': round(current_profit, 2),
                'projected_profit': round(projected_profit, 2),
                'profit_increase': round(projected_profit - current_profit, 2)
            }

        return base_rec

    def get_bundling_recommendations(self, item_name):
        """Get bundling recommendations based on association rules"""
        addon_assoc = self.associations['addon']
        item_assoc = self.associations['item']

        recommendations = {
            'high_confidence_bundles': [],
            'complementary_items': [],
            'add_on_suggestions': []
        }

        # Find item-to-item associations
        item_pairs = item_assoc[item_assoc['item_a'] == item_name].sort_values(
            'confidence', ascending=False
        ).head(5)

        if len(item_pairs) > 0:
            for _, row in item_pairs.iterrows():
                recommendations['complementary_items'].append({
                    'item': row['item_b'],
                    'confidence': f"{row['confidence']*100:.1f}%",
                    'support': f"{row['support']*100:.2f}%"
                })

        # Get top add-on recommendations
        top_addons = addon_assoc.nlargest(5, 'lift')
        for _, row in top_addons.iterrows():
            recommendations['add_on_suggestions'].append({
                'from': row['antecedent'],
                'to': row['consequent'],
                'lift': round(row['lift'], 2),
                'confidence': f"{row['confidence']*100:.1f}%"
            })

        return recommendations

    def generate_menu_report(self, items_to_analyze):
        """
        Generate comprehensive menu engineering report

        Parameters:
        -----------
        items_to_analyze : list of dict
            List of items with keys: item_name, price, food_cost, purchases, desc_length
        """
        print("\n" + "="*80)
        print("MENU ENGINEERING ANALYSIS REPORT")
        print("="*80)
        print(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Items Analyzed: {len(items_to_analyze)}")
        print("="*80)

        results = []
        for item in items_to_analyze:
            analysis = self.analyze_single_item(item)
            if analysis:
                results.append(analysis)

        # Create summary DataFrame
        df_results = pd.DataFrame(results)

        # Category distribution
        print("\n CATEGORY DISTRIBUTION")
        print("-" * 80)
        category_counts = df_results['category'].value_counts()
        for cat, count in category_counts.items():
            print(f"  {cat:15s}: {count:2d} items ({count/len(df_results)*100:.1f}%)")

        # Stars
        stars = df_results[df_results['category'] == 'Star']
        if len(stars) > 0:
            print("\n STARS (High Popularity, High Profit)")
            print("-" * 80)
            for _, item in stars.iterrows():
                print(f"  • {item['item_name']}")
                print(f"    Contribution Margin: ${item['contribution_margin']:.2f}")
                print(f"    Popularity: {item['popularity']:.2f}%")

        # Plowhorses
        plowhorses = df_results[df_results['category'] == 'Plowhorse']
        if len(plowhorses) > 0:
            print("\n PLOWHORSES (High Popularity, Low Profit)")
            print("-" * 80)
            for _, item in plowhorses.iterrows():
                print(f"  • {item['item_name']}")
                print(f"    Current Price: ${item['price']:.2f}")
                rec = item['recommendation']
                if 'price_simulation' in rec:
                    sim = rec['price_simulation']
                    print(f"     Suggested Price: ${sim['suggested_price']:.2f}")
                    print(f"   Profit Increase: ${sim['profit_increase']:.2f}")

        # Puzzles
        puzzles = df_results[df_results['category'] == 'Puzzle']
        if len(puzzles) > 0:
            print("\n PUZZLES (Low Popularity, High Profit)")
            print("-" * 80)
            for _, item in puzzles.iterrows():
                print(f"  • {item['item_name']}")
                print(f"      High margin but underperforming - needs promotion")

        # Dogs
        dogs = df_results[df_results['category'] == 'Dog']
        if len(dogs) > 0:
            print("\n DOGS (Low Popularity, Low Profit)")
            print("-" * 80)
            for _, item in dogs.iterrows():
                print(f"  • {item['item_name']}")
                print(f"    Consider removing or complete overhaul")

        print("\n" + "="*80)
        print("END OF REPORT")
        print("="*80 + "\n")

        return df_results

    def export_recommendations_json(self, results_df, filename='menu_recommendations.json'):
        """Export recommendations to JSON for easy integration"""
        recommendations = []

        for _, row in results_df.iterrows():
            recommendations.append({
                'item_name': row['item_name'],
                'category': row['category'],
                'metrics': {
                    'price': float(row['price']),
                    'food_cost': float(row['food_cost']),
                    'contribution_margin': float(row['contribution_margin']),
                    'food_cost_percentage': float(row['food_cost_percentage']),
                    'popularity': float(row['popularity']),
                    'profit_factor': float(row['profit_factor'])
                },
                'recommendation': row['recommendation']
            })

        output = {
            'generated_at': datetime.now().isoformat(),
            'total_items': len(recommendations),
            'recommendations': recommendations
        }

        with open(filename, 'w') as f:
            json.dump(output, f, indent=2)

        print(f"✓ Recommendations exported to {filename}")
        return filename





In [6]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import json
from datetime import datetime
import warnings


URL_DESC = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis/Datasets/menu_engineering_desc_cleaned.csv"
URL_DOGS = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis/Datasets/menu_engineering_dogs.csv"
URL_INPUT = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis/Datasets/menu_engineering_input_items.csv"
URL_ADDON_ASSOC = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/item_addon_association.csv"
URL_ITEM_ASSOC = "https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/item_item_associations.csv"

warnings.filterwarnings('ignore')

class MenuEngineeringSystem:
    def __init__(self):
        self.model = DecisionTreeClassifier(max_depth=5, random_state=42)
        self.label_encoder = LabelEncoder()
        self.menu_items = pd.DataFrame()
        self.associations = {}
        self.trained = False

    def load_data_from_files(self):
        print("Loading data from GitHub repositories...")
        try:
            # 1. Load Primary Analysis Data
            self.menu_items = pd.read_csv(URL_INPUT)

            # 2. Load Association Logic
            df_item_assoc = pd.read_csv(URL_ITEM_ASSOC)
            df_addon_assoc = pd.read_csv(URL_ADDON_ASSOC)

            # 3. Load Description Data and merge to get desc_length
            df_desc = pd.read_csv(URL_DESC)
            self.menu_items = self.menu_items.merge(
                df_desc[['id', 'desc_length']],
                left_on='item_id',
                right_on='id',
                how='left'
            ).fillna(0)

            self.associations = {
                'addon': df_addon_assoc,
                'item': df_item_assoc
            }

            print(f"✓ Success: Loaded {len(self.menu_items)} menu items.")
            return True
        except Exception as e:
            print(f"✗ Error loading data: {e}")
            return False

    def train_model(self):
        print("\nTraining ML model...")
        if self.menu_items.empty:
            print("✗ No data found.")
            return False

        # Features from your business plan: Price, Purchases, and Description Length
        # We ensure we only use numeric columns for training
        train_df = self.menu_items.dropna(subset=['category'])
        features = ['price', 'purchases', 'desc_length']
        X = train_df[features]
        y = train_df['category']

        y_encoded = self.label_encoder.fit_transform(y)
        X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

        self.model.fit(X_train, y_train)
        self.trained = True

        accuracy = accuracy_score(y_test, self.model.predict(X_test))
        print(f"✓ Model trained. Accuracy: {accuracy*100:.2f}%")
        return True

    def analyze_single_item(self, item_data):
        """Analyzes an item and returns a full strategic profile."""
        if not self.trained: return None

        # Predict Category
        features = np.array([[item_data['price'], item_data['purchases'], item_data.get('desc_length', 0)]])
        category_encoded = self.model.predict(features)[0]
        category = self.label_encoder.inverse_transform([category_encoded])[0]

        # Strategic Rules
        recommendations = {
            'Star': "PROTECT: High profit. Keep in the 'Sweet Spot' (top-right).",
            'Plowhorse': "RE-ENGINEER: High popularity but low margin. Raise price or reduce cost.",
            'Puzzle': "PROMOTE: High profit but low volume. Simplify description to solve paradox.",
            'Dog': "ELIMINATE: Remove or replace with a new concept."
        }

        # Multi-Item Recommendation Logic
        # Finds item_b where item_a is the current item
        item_name = item_data['item_name']
        assoc_matches = self.associations['item'][self.associations['item']['item_a'] == item_name]
        top_recs = assoc_matches.sort_values('confidence', ascending=False).head(2)['item_b'].tolist()

        return {
            'item_name': item_name,
            'predicted_category': category,
            'owner_instruction': recommendations.get(category, "Review performance"),
            'description_paradox_fix': "Shorten description" if item_data.get('desc_length', 0) > 50 else "Optimal",
            'customer_bundle': f"Customers also bought: {', '.join(top_recs)}" if top_recs else "Try our Chef's Special!"
        }

    def export_json(self, analysis_list, filename='menu_output.json'):
        output = {
            "timestamp": datetime.now().isoformat(),
            "data": analysis_list
        }
        with open(filename, 'w') as f:
            json.dump(output, f, indent=4)
        print(f"✓ JSON Exported: {filename}")

# --- MAIN EXECUTION ---
if __name__ == "__main__":
    system = MenuEngineeringSystem()
    if system.load_data_from_files() and system.train_model():

        # Test with items from your actual data
        sample_to_analyze = [
            {'item_name': 'The Classic', 'price': 110.0, 'purchases': 30, 'desc_length': 106},
            {'item_name': 'The Double', 'price': 150.0, 'purchases': 6, 'desc_length': 77},
            {'item_name': 'Hawaii', 'price': 85.0, 'purchases': 15, 'desc_length': 45}
        ]

        final_results = []
        print("\n--- APPLICATION REAL-TIME OUTPUT ---")
        for item in sample_to_analyze:
            result = system.analyze_single_item(item)
            final_results.append(result)
            print(f"\nITEM: {result['item_name']}")
            print(f"OWNER: {result['owner_instruction']}")
            print(f"UI: {result['customer_bundle']}")

        system.export_json(final_results)

Loading data from GitHub repositories...
✓ Success: Loaded 87713 menu items.

Training ML model...
✓ Model trained. Accuracy: 98.83%

--- APPLICATION REAL-TIME OUTPUT ---

ITEM: The Classic
OWNER: PROTECT: High profit. Keep in the 'Sweet Spot' (top-right).
UI: Customers also bought: The Rudimental, The Rudimental Meal

ITEM: The Double
OWNER: PROMOTE: High profit but low volume. Simplify description to solve paradox.
UI: Customers also bought: The Double Meal (V,VG), The Real Deal Meal (V,VG)

ITEM: Hawaii
OWNER: PROTECT: High profit. Keep in the 'Sweet Spot' (top-right).
UI: Customers also bought: Margherita, Vesuvio
✓ JSON Exported: menu_output.json


In [ ]:
import pandas as pd
import json


df_desc = pd.read_csv("https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis/Datasets/menu_engineering_desc_cleaned.csv")
df_items = pd.read_csv("https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/Data%20Analysis/Datasets/menu_engineering_input_items.csv")
df_whatif = pd.read_csv("https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/item_addon_association.csv") # Assuming simulation data is here
df_item_assoc = pd.read_csv("https://raw.githubusercontent.com/Radwa-Ayman239/MenueEngineering/refs/heads/main/item_item_associations.csv")

# Merge Datasets for a unified view
df_main = df_items.merge(df_desc[['id', 'desc_length']], left_on='item_id', right_on='id', how='left')
# For simulation logic, we assume profit_price_up exists or is calculated
df_main['profit_price_up'] = df_main['profit'] * 1.05
df_main = df_main.fillna(0)

# Identify 'Star' items for bundling
star_items = df_main[df_main['category'] == 'Star']['item_name'].tolist()

owner_report = []
customer_menu = []

for _, row in df_main.iterrows():
    item_name = row['item_name']
    category = row['category']

    # --- FILE 1: OWNER STRATEGY JSON ---
    owner_entry = {
        "item_id": int(row['item_id']),
        "item_name": item_name,
        "classification": category,
        "strategic_action": {
            "Star": "Highlight: Keep in the 'Sweet Spot' (Top-Right).",
            "Plowhorse": f"Re-engineer: Price up by 5% to gain ${round(row['profit_price_up'] - row['profit'], 2)} profit.",
            "Puzzle": "Marketing: Move to top and bundle with a Star.",
            "Dog": "Eliminate: Remove or replace with new concept."
        }.get(category, "Monitor"),
        "description_audit": {
            "current_length": int(row['desc_length']),
            "paradox_alert": row['desc_length'] > 50,
            "instruction": "Cut characters by 50% to reduce decision fatigue" if row['desc_length'] > 50 else "Optimal"
        }
    }
    owner_report.append(owner_entry)

    # --- FILE 2: CUSTOMER MENU JSON (Multi-Item Bundling) ---
    # Find all items associated with this one that are also STARS
    assoc_matches = df_item_assoc[df_item_assoc['item_a'] == item_name]
    multi_recs = assoc_matches[assoc_matches['item_b'].isin(star_items)].sort_values('confidence', ascending=False).head(2)
    bundle_suggestions = multi_recs['item_b'].tolist()

    customer_entry = {
        "item_name": item_name,
        "price": row['price'],
        "clean_description": row['description'] if row['desc_length'] < 60 else f"Our signature {item_name}.",
        "upsell_bundle": {
            "message": f"Complete your meal with {', '.join(bundle_suggestions)}!" if bundle_suggestions else "Try our Chef's Choice Star item!",
            "suggested_ids": bundle_suggestions
        }
    }
    customer_menu.append(customer_entry)

# Exporting
with open('owner_recommendations.json', 'w') as f:
    json.dump(owner_report, f, indent=4)

with open('customer_menu_api.json', 'w') as f:
    json.dump(customer_menu, f, indent=4)

print("Files 'owner_recommendations.json' and 'customer_menu_api.json' are ready.")